In [ ]:
# 五感，api容易断，建议每次跑出来的结果分开存储，并且最后合并
# 这里的结果是源文件所有列+“第*行第*列”所有列
import openai
import pandas as pd
import json
import time
import re

openai.api_key = 'sk-kZP3M2vi1WX4JTXs1zZFd9baGGRYPKj5cgZMt64sxNiNvjf1'
openai.api_base = "https://api.chatanywhere.com.cn/v1"

# 匹配模式
pattern = r'(第[一二三四五六七八九十\d]+行)?第([一二三四五六七八九十\d]+)列：([^第。]+)(?=。|第|$)'

def read_excel_data(file_path):
    df = pd.read_excel(file_path)
    return df

# openai key使用
def call_openai(prompt):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",
        prompt=prompt,
        max_tokens=3000,
        temperature=0.6,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].text

# 正则化匹配
def extract_column_content(text):
    matches = re.findall(pattern, text)
    

    row_data = {}
    
    for match in matches:
        row, col, content = match
        row = row if row else "第一行" 
        col = f"第{col}列"
        

        if row not in row_data:
            row_data[row] = {}

        row_data[row][col] = content.strip()

    df = pd.DataFrame(row_data).T
    return df

with open('./五感_建议3.prompt', 'r', encoding='utf-8') as f:
    k = f.read()

file_path = '苏泊尔低价格段_使用前.xlsx'
df = read_excel_data(file_path) 
comments = df['text'].tolist()
responses = []

p = f"现在，请帮我分析以下评论内容，根据用户评论进行分析.：\n"
r = " "
count = 0
for comment in comments:
    input_item = k + p + comment + r
    rsp = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[ 
            {"role": "system", "content": "一个很懂电商用户心理的AI"},
            {"role": "user", "content": input_item}
        ],
        max_tokens=3000,
        temperature=0.6,
    )
    generated_response = rsp['choices'][0]['message']['content']
    print(generated_response)

    column_df = extract_column_content(generated_response)

    response_row = df.loc[df['text'] == comment].to_dict(orient='records')[0]
    response_row.update({'AI分析': generated_response})
    
    for row_id, columns in column_df.iterrows():
        for col_id, value in columns.dropna().items():
            response_row[f"{row_id}-{col_id}"] = value

    responses.append(response_row)
    
    count += 1
    print(count)
    time.sleep(0.1)

response_df = pd.DataFrame(responses)
response_df.to_csv('苏泊尔低价格段_使用前_五感初始_1.csv', index=False, encoding='utf-8-sig')


In [ ]:
# 从前面的格式中把“第一列”-“第五列”分别映射到"用户评论", "情感倾向", "需求模块", "产品功能模块", "用户期望"
import pandas as pd

# 读取CSV文件
df = pd.read_csv('竞品中价格段_使用中_五感初始合并后.csv', encoding='utf-8-sig')  # 请替换为你的实际文件路径

# 打印列名，确认其结构
print("DataFrame中的列名:", df.columns.tolist())

# 确定静态列的范围
try:
    static_columns = list(df.columns[:df.columns.get_loc("第一行-第一列")])
except KeyError:
    print("错误: 找不到 '第一行-第一列' 列。请检查列名是否正确。")
    static_columns = []

# 创建一个映射字典，将动态列映射到新的属性名
dynamic_column_mapping = {
    "第*行-第*列": [
        "用户评论", "情感倾向", "需求模块", "产品功能模块", "用户期望"
    ]
}

# 分解动态列为独立行
output_rows = []
for _, row in df.iterrows():
    # 获取静态列的数据
    static_data = row[static_columns].to_dict()
    
    # 遍历"第一行", "第二行", 等等
    for i in ["一", "二", "三", "四","五","六","七","八","九","十","十一","十二","十三","十四","十五","十六","十七","十八","十九","二十","二十一","二十二","二十三","二十四","二十五"]:
        # 为每个分组生成动态列名 (例如，第一行-第一列 到 第一行-第五列)
        dynamic_columns = [f"第{i}行-第{j}列" for j in ["一", "二", "三", "四", "五"]]
        
        # 过滤掉DataFrame中不存在的列
        valid_dynamic_columns = [col for col in dynamic_columns if col in df.columns]
        
        if not valid_dynamic_columns:
            #print(f"未找到第{i}行的列。")
            continue
        
        # 检查当前组是否有非空值
        dynamic_data = row[valid_dynamic_columns].dropna()
        if not dynamic_data.empty:
            # 将静态数据和动态数据映射到新的列名
            new_row = {**static_data}

            # 遍历动态数据，并将其映射到新的列名
            for idx, dynamic_value in enumerate(dynamic_data):
                new_row[dynamic_column_mapping["第*行-第*列"][idx]] = dynamic_value

            output_rows.append(new_row)

# 创建一个新的DataFrame存储扩展后的行
expanded_df = pd.DataFrame(output_rows)

# 保存扩展后的数据到新的CSV文件
expanded_df.to_csv('竞品中价格段_使用中_五感诊断.csv', index=False, encoding='utf-8-sig')
